# Тестирование хранилищ (MongoDB и ClickHouse)
### Импорт библиотек

In [1]:
import os
import json
import statistics
import time

from clickhouse_driver import Client
from dotenv import load_dotenv
from pymongo import MongoClient

### Используемые функции

In [34]:
def benchmark(func):
    """
    Декоратор функции, возвращает среднее время выполнения функции в цикле из 100 повторений
    """
    def wrapper(*args):
        result = None
        list_time = []
        for attempt in range(100):
            start = time.monotonic()
            result = func(*args)
            end = time.monotonic()
            list_time.append(end - start)
        avg = statistics.mean(list_time)
        print(f"--- Среднее выполнение скрипта {avg} секунд ---")
        print(result)
    return wrapper

def benchmark_click(func):
    """
    Декоратор функции, возвращает время выполнения функции
    """
    def wrapper(*args):
        start = time.monotonic()
        func(*args)
        end = time.monotonic()
        tm = end - start
        print(f"--- Выполнение скрипта {tm} секунд ---")
    return wrapper

def insert_doc(collection, data: dict):
    """
    Функция вставляет один документ в коллекцию MongoDB и возвращает его id
    """
    return collection.insert_one(data).inserted_id

def insert_docs(collection, data: list):
    """
    Функция вставляет список документов в коллекцию MongoDB
    """
    return collection.insert_many(data)

def find_doc(collection, elements: dict, multiple=False):
    """
    Функция возвращает один или список документов из коллекции MongoDB.
    """
    if multiple:
        results = collection.find(elements)
        return [r for r in results]
    else:
        return collection.find_one(elements)

def count_docs(collection, elements: dict):
    """
    Функция возвращает количество документов из коллекции MongoDB по условию.
    """
    result = collection.count_documents(elements)
    return result

def find_doc_like(collection, elements: dict, field: str, num: int):
    """
    Функция возвращает список фильмов с лайками пользователя из коллекции MongoDB.
    """
    results = collection.find({"$and" : [elements, {field: {"$gte": num}}]})
    return [r for r in results]

def count_doc_like(collection, elements: dict, field: str, num: int):
    """
    Функция возвращает количество лайков у фильма из коллекции MongoDB.
    """
    results = collection.count_documents({"$and" : [elements, {field: {"$gte": num}}]})
    return results

def count_doc_dislike(collection, elements: dict, field: str, num: int):
    """
    Функция возвращает количество дизлайков у фильма из коллекции MongoDB.
    """
    results = collection.count_documents({"$and" : [elements, {field: {"$lt": num}}]})
    return results


@benchmark
def test_avg_like(collection, conditions: list):
    """
    Функция возвращает среднюю оценку фильма.
    Тестирование времени выполнения скрипта.
    """
    return list(collection.aggregate(conditions))

@benchmark
def test_find_doc(collection, elements: dict, multiple: bool):
    """
    Функция возвращает список закладок или одну закладку пользователя.
    Тестирование времени работы функции
    """
    return find_doc(collection, elements, multiple)

@benchmark
def test_count_docs(collection, elements: dict):
    """
    Функция возвращает количество документов в коллекции MongoDB
    Тестирование времени работы
    """
    return count_docs(collection, elements)

@benchmark
def test_find_doc_like(collection, elements: dict, field: str, num: int):
    """
    Функция возвращает список пользователей, которым понравился фильм.
    Тестирование времени работы
    """
    return find_doc_like(collection, elements, field, num)

@benchmark
def test_count_doc_like(collection, elements: dict, field: str, num: int):
    """
    Функция возвращает количество лайков у фильма.
    Тестирование времени работы
    """
    return count_doc_like(collection, elements, field, num)

@benchmark
def test_count_doc_dislike(collection, elements: dict, field: str, num: int):
    """
    Функция возвращает количество дизлайков у фильма.
    Тестирование времени работы
    """
    return count_doc_dislike(collection, elements, field, num)

@benchmark_click
def upload_likes_clickhouse(client_click: Client, data: str):
    """
    Загрузка данных таблицы likes в ClickHouse
    Тестирование времени работы
    """
    return client_click.execute(
                """
                INSERT INTO movies.likes (
                movie_id, user_id, "like")  VALUES {}
                """.format(
                    data
                )
            )

@benchmark_click
def upload_bookmarks_clickhouse(client_click: Client, data: str):
    """
    Загрузка данных таблицы bookmarks в ClickHouse
    Тестирование времени работы
    """
    return client_click.execute(
                """
                INSERT INTO movies.bookmarks (
                user_id, movie_id)  VALUES {}
                """.format(
                    data
                )
            )

@benchmark_click
def upload_reviews_clickhouse(client_click: Client, data: str):
    """
    Загрузка данных таблицы reviews в ClickHouse
    Тестирование времени работы
    """
    return client_click.execute(
                """
                INSERT INTO movies.reviews (
                movie_id, user_id, "like", rating_movie, text)  VALUES {}
                """.format(
                    data
                )
            )

@benchmark
def test_find_doc_click(client_click: Client, user_id: dict):
    """
    Функция возвращает список закладок пользователя.
    Тестирование времени работы функции
    """
    return client_click.execute(f"""
                SELECT *
                FROM movies.bookmarks
                WHERE user_id=%(user_id)s
            """, user_id)

@benchmark
def test_count_doc_click(client_click: Client, user_id: dict):
    """
    Функция возвращает количество закладок пользователя.
    Тестирование времени работы функции
    """
    return client_click.execute(f"""
                SELECT count(user_id)
                FROM movies.bookmarks
                WHERE user_id=%(user_id)s
                group by user_id
            """, user_id)

@benchmark
def test_avg_movie_click(client_click: Client, movie_id: dict):
    """
    Функция возвращает среднюю оценку фильма.
    Тестирование времени выполнения скрипта.
    """
    return client_click.execute("""
                SELECT avg(like)
                FROM movies.likes
                WHERE movie_id=%(movie_id)s
                group by movie_id
            """, movie_id
    )

## MongoDB

### Переменные

In [3]:
load_dotenv()

MONGO_HOST = os.getenv("MONGO_HOST")
MONGO_PORT = int(os.getenv("MONGO_PORT"))
DB_NAME = os.getenv("MONGO_INITDB_DATABASE")
USERNAME = os.getenv("ME_CONFIG_MONGODB_AUTH_USERNAME")
PASSWORD = os.getenv("ME_CONFIG_MONGODB_AUTH_PASSWORD")

### Подключаемся к Mongo

In [4]:
client = MongoClient(
    MONGO_HOST,
    MONGO_PORT,
    username=USERNAME,
    password=PASSWORD)
db = client.get_database(DB_NAME)

BOOKMARKS = db.get_collection("bookmarks")
LIKES = db.get_collection("likes")
REVIEWS = db.get_collection("reviews")

### Читаем лайки

In [5]:
with open("data/likes.csv", 'r') as file:
    likes_str = file.read().splitlines()

In [6]:
likes = []
for i in range(len(likes_str)):
    like = json.loads(likes_str[i].replace('"', "'").replace("'", '"'))
    likes.append(like)
print('список с лайками подготовлен')

список с лайками подготовлен


#### Добавляем в MongoDB

In [7]:
start_time = time.time()
insert_docs(LIKES, likes)
len_likes = len(likes)
print(f"Готово. В Mongo внесено {len_likes} записей.")
print("--- Выполнение скрипта %s секунд ---" % (time.time() - start_time))

Готово. В Mongo внесено 4161834 записей.
--- Выполнение скрипта 136.1855113506317 секунд ---


# Читаем закладки

In [8]:
with open("data/bookmarks.csv", 'r') as file:
    bookmarks_str = file.read().splitlines()
bookmarks = []
for i in range(len(bookmarks_str)):
    bookmark = json.loads(bookmarks_str[i].replace('"', "'").replace("'", '"'))
    bookmarks.append(bookmark)
print('список с закладками подготовлен')

список с закладками подготовлен


#### Добавляем в MongoDB

In [9]:
start_time = time.time()
insert_docs(BOOKMARKS, bookmarks)
len_bookmarks = len(bookmarks)
print(f"Готово. В Mongo внесено {len_bookmarks} записей.")
print("--- Выполнение скрипта %s секунд ---" % (time.time() - start_time))

Готово. В Mongo внесено 2079804 записей.
--- Выполнение скрипта 51.79152464866638 секунд ---


# Читаем рецензии

In [10]:
with open("data/reviews.csv", 'r') as file:
    reviews_str = file.read().splitlines()
reviews = []
for i in range(len(reviews_str)):
    review = json.loads(reviews_str[i].replace('"', "'").replace("'", '"'))
    reviews.append(review)
print('список с рецензиями подготовлен')

список с рецензиями подготовлен


In [11]:
#### Добавляем в MongoDB

In [12]:
start_time = time.time()
insert_docs(REVIEWS, reviews)
len_reviews = len(reviews)
print(f"Готово. В Mongo внесено {len_reviews} записей.")
print("--- Выполнение скрипта %s секунд ---" % (time.time() - start_time))

Готово. В Mongo внесено 4161834 записей.
--- Выполнение скрипта 189.1337969303131 секунд ---


## Тестирование

### список закладок

In [13]:
test_find_doc(BOOKMARKS, {'user_id': 'abe45b0b-444f-4da9-8c7e-6d8a2341422f'}, True)

--- Среднее выполнение скрипта 0.004035453796386718 секунд ---
[{'_id': ObjectId('6262c3acfd421c33405006ac'), 'user_id': 'abe45b0b-444f-4da9-8c7e-6d8a2341422f', 'movie_id': '00e2e781-7af9-4f82-b4e9-14a488a3e184'}, {'_id': ObjectId('6262c3acfd421c33405006ad'), 'user_id': 'abe45b0b-444f-4da9-8c7e-6d8a2341422f', 'movie_id': '025c58cd-1b7e-43be-9ffb-8571a613579b'}, {'_id': ObjectId('6262c3acfd421c33405006ae'), 'user_id': 'abe45b0b-444f-4da9-8c7e-6d8a2341422f', 'movie_id': '027ba6c5-d805-402a-addf-484b34292625'}, {'_id': ObjectId('6262c3acfd421c33405006af'), 'user_id': 'abe45b0b-444f-4da9-8c7e-6d8a2341422f', 'movie_id': '0312ed51-8833-413f-bff5-0e139c11264a'}, {'_id': ObjectId('6262c3acfd421c33405006b0'), 'user_id': 'abe45b0b-444f-4da9-8c7e-6d8a2341422f', 'movie_id': '03b0c429-a63f-4045-a485-538a36c89264'}, {'_id': ObjectId('6262c3acfd421c33405006b1'), 'user_id': 'abe45b0b-444f-4da9-8c7e-6d8a2341422f', 'movie_id': '044beafe-fe25-4edc-95f4-adbb8979c35b'}, {'_id': ObjectId('6262c3acfd421c3340

### Количество закладок

In [14]:
test_count_docs(BOOKMARKS, {'user_id': 'abe45b0b-444f-4da9-8c7e-6d8a2341422f'})

--- Среднее выполнение скрипта 0.0011175107955932618 секунд ---
501


### Список всех пользователей, поставивших лайк фильму


In [15]:
test_find_doc(LIKES, {'movie_id': '0f05c4a6-7fc6-4b2a-9ecc-551979316aea'}, True)

--- Среднее выполнение скрипта 0.03127215147018433 секунд ---
[{'_id': ObjectId('6262c315fd421c3340fb4e8e'), 'movie_id': '0f05c4a6-7fc6-4b2a-9ecc-551979316aea', 'user_id': '0031feab-8f53-412a-8f53-47098a60ac73', 'like': 9}, {'_id': ObjectId('6262c315fd421c3340fb5275'), 'movie_id': '0f05c4a6-7fc6-4b2a-9ecc-551979316aea', 'user_id': '00395304-dd52-4c7b-be0d-c2cd7a495684', 'like': 3}, {'_id': ObjectId('6262c315fd421c3340fb565c'), 'movie_id': '0f05c4a6-7fc6-4b2a-9ecc-551979316aea', 'user_id': '0041c8a3-345b-43c8-a615-4d8d62005d4b', 'like': 4}, {'_id': ObjectId('6262c315fd421c3340fb5a43'), 'movie_id': '0f05c4a6-7fc6-4b2a-9ecc-551979316aea', 'user_id': '00591223-0fd4-4a5f-89f8-0317eff58c1b', 'like': 8}, {'_id': ObjectId('6262c315fd421c3340fb5e2a'), 'movie_id': '0f05c4a6-7fc6-4b2a-9ecc-551979316aea', 'user_id': '006d10a0-b0ba-4dac-bcf0-1111411c178b', 'like': 6}, {'_id': ObjectId('6262c315fd421c3340fb6211'), 'movie_id': '0f05c4a6-7fc6-4b2a-9ecc-551979316aea', 'user_id': '0092a672-4ed5-4838-a6f

### Количество фильмов

In [16]:
test_count_docs(LIKES, {'movie_id': '0f05c4a6-7fc6-4b2a-9ecc-551979316aea'})

--- Среднее выполнение скрипта 0.0024084043502807617 секунд ---
4166


### Средняя пользовательская оценка фильма

In [17]:
pipeline = [
    { "$match": { 'movie_id': "0d03e753-cec7-4822-81f8-ec0499fb09d7" } },
   {
     "$group": {
        "_id": "null",
        "total": { "$avg": "$like" }
     }
   }
]

test_avg_like(LIKES, pipeline)

--- Среднее выполнение скрипта 0.01644974946975708 секунд ---
[{'_id': 'null', 'total': 4.43638982237158}]


### Кому из пользователей понравился фильм

In [18]:
test_find_doc_like(LIKES, {"movie_id": "0f05c4a6-7fc6-4b2a-9ecc-551979316aea"}, "like", 5)

--- Среднее выполнение скрипта 0.022176973819732666 секунд ---
[{'_id': ObjectId('6262c315fd421c3340fb4e8e'), 'movie_id': '0f05c4a6-7fc6-4b2a-9ecc-551979316aea', 'user_id': '0031feab-8f53-412a-8f53-47098a60ac73', 'like': 9}, {'_id': ObjectId('6262c315fd421c3340fb5a43'), 'movie_id': '0f05c4a6-7fc6-4b2a-9ecc-551979316aea', 'user_id': '00591223-0fd4-4a5f-89f8-0317eff58c1b', 'like': 8}, {'_id': ObjectId('6262c315fd421c3340fb5e2a'), 'movie_id': '0f05c4a6-7fc6-4b2a-9ecc-551979316aea', 'user_id': '006d10a0-b0ba-4dac-bcf0-1111411c178b', 'like': 6}, {'_id': ObjectId('6262c315fd421c3340fb65f8'), 'movie_id': '0f05c4a6-7fc6-4b2a-9ecc-551979316aea', 'user_id': '009884ca-5a33-44ba-8fba-495a0055c70d', 'like': 5}, {'_id': ObjectId('6262c315fd421c3340fb69df'), 'movie_id': '0f05c4a6-7fc6-4b2a-9ecc-551979316aea', 'user_id': '009a900e-b9dc-4cd4-87a7-ca53d1b7dd24', 'like': 9}, {'_id': ObjectId('6262c315fd421c3340fb71ad'), 'movie_id': '0f05c4a6-7fc6-4b2a-9ecc-551979316aea', 'user_id': '00a770fc-8964-41fa-9d

### Количество лайков у определённого фильма

In [19]:
test_count_doc_like(LIKES, {"movie_id": "0f05c4a6-7fc6-4b2a-9ecc-551979316aea"}, "like", 5)

--- Среднее выполнение скрипта 0.014572832584381103 секунд ---
2105


### Количество дизлайков у определённого фильма

In [20]:
test_count_doc_dislike(LIKES, {"movie_id": "0f05c4a6-7fc6-4b2a-9ecc-551979316aea"}, "like", 5)

--- Среднее выполнение скрипта 0.014313299655914307 секунд ---
2061


### Список понравившихся пользователю фильмов

In [21]:
test_find_doc_like(LIKES, {"user_id": "d8cf8c23-72f8-4ac0-87b4-85c5b2920f95"}, "like", 5)

--- Среднее выполнение скрипта 0.004707908630371094 секунд ---
[{'_id': ObjectId('6262c329fd421c33403186f5'), 'movie_id': '00e2e781-7af9-4f82-b4e9-14a488a3e184', 'user_id': 'd8cf8c23-72f8-4ac0-87b4-85c5b2920f95', 'like': 8}, {'_id': ObjectId('6262c329fd421c33403186f7'), 'movie_id': '01cd80e2-5db8-4914-9a80-74f15a3a1a24', 'user_id': 'd8cf8c23-72f8-4ac0-87b4-85c5b2920f95', 'like': 5}, {'_id': ObjectId('6262c329fd421c33403186f8'), 'movie_id': '01f81c66-d968-4375-bbb0-65103aa214d1', 'user_id': 'd8cf8c23-72f8-4ac0-87b4-85c5b2920f95', 'like': 9}, {'_id': ObjectId('6262c329fd421c33403186f9'), 'movie_id': '020adfa7-7251-4fb9-b6db-07b60664cb67', 'user_id': 'd8cf8c23-72f8-4ac0-87b4-85c5b2920f95', 'like': 5}, {'_id': ObjectId('6262c329fd421c33403186fb'), 'movie_id': '025c58cd-1b7e-43be-9ffb-8571a613579b', 'user_id': 'd8cf8c23-72f8-4ac0-87b4-85c5b2920f95', 'like': 5}, {'_id': ObjectId('6262c329fd421c33403186fc'), 'movie_id': '027ba6c5-d805-402a-addf-484b34292625', 'user_id': 'd8cf8c23-72f8-4ac0-87

### Количество понравившихся пользователю фильмов

In [22]:
test_count_doc_like(LIKES, {"user_id": "d8cf8c23-72f8-4ac0-87b4-85c5b2920f95"}, "like", 5)

--- Среднее выполнение скрипта 0.002350332736968994 секунд ---
513


## ClickHouse


### Подключаемся к ClickHouse

In [23]:
client = Client(host='127.0.0.1')

### Создаем таблицу likes

In [24]:
client.execute(
        """CREATE TABLE IF NOT EXISTS movies.likes ON CLUSTER company_cluster(
            movie_id String,
            user_id String,
            "like" Int8
            ) Engine=MergeTree()
            ORDER BY movie_id;
     """)
print('Таблица likes создана.')

Таблица likes создана.


### Создаем таблицу bookmarks

In [25]:
client.execute(
        """CREATE TABLE IF NOT EXISTS movies.bookmarks ON CLUSTER company_cluster(
            user_id String,
            movie_id String
            ) Engine=MergeTree()
            ORDER BY user_id;
     """)
print('Таблица likes создана.')

Таблица likes создана.


### Создаем таблицу reviews

In [26]:
client.execute(
        """CREATE TABLE IF NOT EXISTS movies.reviews ON CLUSTER company_cluster(
            movie_id String,
            user_id String,
            "like" Int8,
            rating_movie Int8,
            text MEDIUMTEXT
            ) Engine=MergeTree()
            ORDER BY movie_id;
     """)
print('Таблица likes создана.')

Таблица likes создана.


### Преобразуем данные для хранилища ClickHouse

In [27]:
likes_ch = []
for i in likes:
    string_sql = f"""(\'{i['movie_id']}\', \'{i['user_id']}\', \'{i['like']}\')"""
    likes_ch.append(string_sql)
likes_ch[:1]

["('00af52ec-9345-4d66-adbe-50eb917f463a', '0031feab-8f53-412a-8f53-47098a60ac73', '0')"]

### Загружаем в ClickHouse Likes

In [28]:
values: str = ', '.join(likes_ch)
upload_likes_clickhouse(client, values)

--- Выполнение скрипта 15.98214864730835 секунд ---


### Преобразуем данные для хранилища ClickHouse

In [29]:
bookmarks_ch = []
for i in bookmarks:
    string_sql = f"""(\'{i['user_id']}\', \'{i['movie_id']}\')"""
    bookmarks_ch.append(string_sql)
bookmarks_ch[:1]

["('0031feab-8f53-412a-8f53-47098a60ac73', '00af52ec-9345-4d66-adbe-50eb917f463a')"]

### Загружаем в ClickHouse bookmarks

In [30]:
values: str = ', '.join(bookmarks_ch)
upload_bookmarks_clickhouse(client, values)

--- Выполнение скрипта 3.5947763919830322 секунд ---


### Преобразуем данные для хранилища ClickHouse

In [31]:
reviews_ch = []
for i in reviews:
    string_sql = f"""(\'{i['movie_id']}\', \'{i['user_id']}\', \'{i['like']}\', \'{i['rating_movie']}\', \'{i['text']}\')"""
    reviews_ch.append(string_sql)
reviews_ch[:1]

["('00af52ec-9345-4d66-adbe-50eb917f463a', '0031feab-8f53-412a-8f53-47098a60ac73', '9', '0', 'Scary movie')"]

### Загружаем в ClickHouse reviews

In [32]:
values: str = ', '.join(reviews_ch)
upload_reviews_clickhouse(client, values)

--- Выполнение скрипта 23.50383710861206 секунд ---


## Чтение
### список закладок

In [35]:
test_find_doc_click(client, {'user_id': 'abe45b0b-444f-4da9-8c7e-6d8a2341422f'})

--- Среднее выполнение скрипта 0.008785707950592041 секунд ---
[('abe45b0b-444f-4da9-8c7e-6d8a2341422f', '00e2e781-7af9-4f82-b4e9-14a488a3e184'), ('abe45b0b-444f-4da9-8c7e-6d8a2341422f', '025c58cd-1b7e-43be-9ffb-8571a613579b'), ('abe45b0b-444f-4da9-8c7e-6d8a2341422f', '027ba6c5-d805-402a-addf-484b34292625'), ('abe45b0b-444f-4da9-8c7e-6d8a2341422f', '0312ed51-8833-413f-bff5-0e139c11264a'), ('abe45b0b-444f-4da9-8c7e-6d8a2341422f', '03b0c429-a63f-4045-a485-538a36c89264'), ('abe45b0b-444f-4da9-8c7e-6d8a2341422f', '044beafe-fe25-4edc-95f4-adbb8979c35b'), ('abe45b0b-444f-4da9-8c7e-6d8a2341422f', '04b4f5a0-843c-42e8-bbaf-405425b221f1'), ('abe45b0b-444f-4da9-8c7e-6d8a2341422f', '0657217e-9efa-48fe-be08-6ca29bcaf042'), ('abe45b0b-444f-4da9-8c7e-6d8a2341422f', '06b2669a-2338-4d9d-9cf1-4e8561f8b1dc'), ('abe45b0b-444f-4da9-8c7e-6d8a2341422f', '07b7923f-7238-4413-89ca-632cc2243156'), ('abe45b0b-444f-4da9-8c7e-6d8a2341422f', '07bad56a-4b46-4dd8-a594-64fac39f4fb0'), ('abe45b0b-444f-4da9-8c7e-6d8a2341

### Количество закладок

In [36]:
test_count_doc_click(client, {'user_id': 'abe45b0b-444f-4da9-8c7e-6d8a2341422f'})

--- Среднее выполнение скрипта 0.0053854846954345705 секунд ---
[(501,)]


## Средняя пользовательская оценка фильма

In [37]:
test_avg_movie_click(client, {'movie_id': '0d03e753-cec7-4822-81f8-ec0499fb09d7'})

--- Среднее выполнение скрипта 0.004554035663604736 секунд ---
[(4.43638982237158,)]


## RealTime


### MongoDB

In [38]:
with open("data/likes_rt.csv", 'r') as file:
    likes_str_rt = file.read().splitlines()
likes_rt = []
for i_rt in range(len(likes_str_rt)):
    like_rt = json.loads(likes_str_rt[i_rt].replace('"', "'").replace("'", '"'))
    likes_rt.append(like_rt)
print('список с лайками подготовлен')
#### Добавляем в MongoDB
start_time = time.time()
insert_docs(LIKES, likes_rt)
len_likes_rt = len(likes_rt)
print(f"Готово. В Mongo внесено {len_likes_rt} записей.")
print("--- Выполнение скрипта %s секунд ---" % (time.time() - start_time))

# Средняя оценка фильма
print('Средняя оценка фильма:')
pipeline_rt = [
    {"$match": {'movie_id': "00000000-0000-0000-0000-000000000000"}},
    {
        "$group": {
            "_id": "null",
            "total": {"$avg": "$like"}
        }
    }
]
test_avg_like(LIKES, pipeline_rt)
print('Количество лайков у определенного фильма:')
# Количество лайков у определенного фильма
test_count_doc_like(LIKES, {"movie_id": "00000000-0000-0000-0000-000000000000"}, "like", 5)

список с лайками подготовлен
Готово. В Mongo внесено 10 записей.
--- Выполнение скрипта 0.026825904846191406 секунд ---
Средняя оценка фильма:
--- Среднее выполнение скрипта 0.001031796932220459 секунд ---
[]
Количество лайков у определенного фильма:
--- Среднее выполнение скрипта 0.0009965968132019042 секунд ---
0


### ClickHouse

In [39]:
with open("data/likes_cl.csv", 'r') as file:
    likes_str_rt_cl = file.read().splitlines()
likes_rt_cl = []
for i in range(len(likes_str_rt_cl)):
    like_rt_cl = json.loads(likes_str_rt_cl[i].replace('"', "'").replace("'", '"'))
    likes_rt_cl.append(like_rt_cl)

likes_ch_rt = []
for i in likes_rt_cl:
    string_sql_rt = f"""(\'{i['movie_id']}\', \'{i['user_id']}\', \'{i['like']}\')"""
    likes_ch_rt.append(string_sql_rt)
print('Данные считаны их файла и подготовлены для загрузки в ClickHouse')
### Загружаем в ClickHouse Likes
print('Загружаем в ClickHouse Likes')
values_rt: str = ', '.join(likes_ch_rt)
upload_likes_clickhouse(client, values_rt)

print('стучимся')

test_avg_movie_click(client, {'movie_id': '20000000-0000-0000-0000-000000000000'})
test_avg_movie_click(client, {'movie_id': '20000000-0000-0000-0000-000000000001'})

Данные считаны их файла и подготовлены для загрузки в ClickHouse
Загружаем в ClickHouse Likes
--- Выполнение скрипта 0.006376743316650391 секунд ---
стучимся
--- Среднее выполнение скрипта 0.004468042850494385 секунд ---
[(4.0,)]
--- Среднее выполнение скрипта 0.0052964377403259275 секунд ---
[(5.6,)]
